# Simple Chatbot using LangGraph

Start by creating a **StateGraph**. We’ll add a node to represent the LLM call:

In [37]:
from typing import Annotated, TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver


class State(TypedDict):
    # Messages have the type "list". The `add_messages` 
    # function in the annottion defines how this state should
    # be updated (in this case, it appends new messages to the
    # list, rather than replacing the previous messages)
    messages: Annotated[list, add_messages]


# The state consists of the shape, or schema, of the graph state, 
# as well as reducer functions that specify how to apply updates to the state
builder = StateGraph(State)

#So now our graph knows two things:

#Every *node* we define will receive the current State as input and return a value
# that updates that state. *messages* will be appended to the current list, rather than directly
# overwritten. This is communicated via the prebuilt add_messages function in the Annotated
# syntax in the Python example or the reducer function for the JavaScript example.

# Nodes represents unit of work

In [38]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21")

def chatbot(state: State):
    answer = model.invoke(state["messages"])
    return {"messages": [answer]}

# The first argument is the unique node name
# The second argument is the function or Runnable to run
builder.add_node("chatbot", chatbot)

# This node receives the current state, does one LLM call, and then returns an update
# to the state containing the new message produced by the LLM. The *add_messages* reducer
# appends this message to the messages already in the state.

In [39]:
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

# memory saver

graph = builder.compile(checkpointer=MemorySaver())
# This returns a runnable object with the same methods as the one
# used in the previous code block. But now, it stores the state at
# the end of each step, so every invocation after the first doesn’t
# start from a blank slate. Any time the graph is called, it starts
# by using the checkpointer to fetch the most recent saved state, if
# any, and combines the new input with the previous state. And only
# then does it execute the first nodes.

# It tells the graph where to start its work each time you run it.

# This instructs the graph where it should exit (this is optional, as LangGraph will stop execution once there’s no more nodes to run).

# It compiles the graph into a runnable object, with the familiar invoke and stream methods.

Show the graph:

In [40]:
graph.get_graph().draw_mermaid_png(output_file_path="../img/graph.png")

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00j\x00\x00\x00\xea\x08\x02\x00\x00\x00\xc5\xf3G\x18\x00\x00\x10\x00IDATx\x9c\xec\x9d\tXSW\xbe\xc0o\xc8B\xf6\x04B\x90]@\x8a\x0b\xa2\xa2 n\xd4\r\xb7q\xab\xd5\xb1jkk\xfb|\x8eK\xdbg;\xd5Q\xdb\xaau\xab\xdfT\xab\xdd\\Zk\xeb\xebhm\xeb\xb8\x8b\xc5\xf6U\xeb\x8a\xa2,V\x11\x11\x10\x90\x1d\x02!;Inx\xff\x90\x9626\xc9M8\t\rp~\x9f\x1f&\xf7\x9c\x9b\xe5\x97\xff=\xf7,\xf7\x9e\xc3hjj"0m\x85A`\x10\xc0\xfa\x90\xc0\xfa\x90\xc0\xfa\x90\xc0\xfa\x90\xc0\xfa\x90@\xd5WY\xa4S+H\x9d\x9a\xd4iH\xd2\xd01\xea@t&\x8d\xcd\xa5\xb3yt\xbe\x88\xde\xad;\x9b@\x80\xd6\xb6z\xdf\xc3;\xea\xc2;\xea\x82\xdb*\x81\x98!\xf4e\xc2Ga\xf3\xbc\x98,/\xa2#`\xd0\x9btj\x93VM*d\x06u\x83\xb1G\x7f~d_^x\x0c\x8fp\x1e\xa7\xf5U?j\xbc\xf0]\xb5\xa1\xd1\xd43^\x185\x80/\x962\x89\x8e\x8c\xbc\xc6\xf0 Sy\xff\xa6\xd2\x9b\xe35\xea\xaf\xfe\xd2\x10o\xa7vwB\x1f\x1c\x9b\x17\x8f\xd6\x14\xe7j\x12\'\xfa\xf6N\x14\x12\x9d\x8b\xbb\xd7\x147\xbe\x97E\xc6\xf2G\xce\x92:\xbe\x97\xa3\xfa\xb4*\xf2\xd4\xa7\xe5PR\x8c\x9c\xe9\xc4\x

Show the image

![Agent diagram](../img/graph.png)

In [44]:
# graph can be run with stream or invoke method
thread1 = {"configurable": {"thread_id": "1"}}
result_1 = graph.invoke(
    {"messages": [HumanMessage("hi, my name is Jack!")]},
    thread1
)
print(result_1)

# Notice the object thread1, which identifies the current interaction
# as belonging to a particular history of interactions—which are called
# threads in LangGraph. Threads are created automatically when first used.
# Any string is a valid identifier for a thread (usually, Universally
# Unique Identifiers [UUIDs] are used).

{'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='301cb710-faf5-4be3-bd45-79407ba727ab'), AIMessage(content='Hi Jack! 👋 Nice to meet you! How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 14, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZVmkvk5LZbQWYDywotFQO2TSrYTj', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'fin

In [45]:
result_2 = graph.invoke(
    {"messages": [HumanMessage("What is my name?")]},
    thread1
)
print(result_2)

{'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='301cb710-faf5-4be3-bd45-79407ba727ab'), AIMessage(content='Hi Jack! 👋 Nice to meet you! How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 14, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZVmkvk5LZbQWYDywotFQO2TSrYTj', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'fin

You can also inspect and update the state directly; let’s see how:



In [46]:
graph.get_state(thread1)

StateSnapshot(values={'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='301cb710-faf5-4be3-bd45-79407ba727ab'), AIMessage(content='Hi Jack! 👋 Nice to meet you! How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 14, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZVmkvk5LZbQWYDywotFQO2TSrYTj', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severi

To update the state you can do the following:

In [ ]:
graph.update_state(config=thread1, values={"message": [HumanMessage('I like LLMs!')]})
# This would add one more message to the list of messages in the state, to be used the next
# time you invoke the graph on this thread.

# show the update:

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f036011-6b25-63b2-800b-bbd764f0a77b'}}

In [49]:
graph.get_state(thread1)

StateSnapshot(values={'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='301cb710-faf5-4be3-bd45-79407ba727ab'), AIMessage(content='Hi Jack! 👋 Nice to meet you! How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 14, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZVmkvk5LZbQWYDywotFQO2TSrYTj', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severi

## Modifying chat history
As previous messages aren't in the best state or format to generate an accurate response from the model.
There are 3 methods:
1. Trimming
2. Filtering
3. Merging

### 

### Trimming

In [1]:
# This basically involves loading and storing the most recent messages. Langchain does offer a trim message method
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage,trim_messages
from langchain_openai import AzureChatOpenAI

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21",model="gpt-4o"),
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's 2 + 2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

#### Key Parameters and Their Descriptions

- **`strategy`**  
  Determines whether trimming should start from the beginning or the end of the list.  
  In most cases, it’s best to preserve the most recent messages and discard older ones that don't fit.  
  To enable this behavior, set `strategy="last"`.  
  Alternatively, use `strategy="first"` to keep the earliest messages and trim newer ones if necessary.

- **`token_counter`**  
  Refers to the language or chat model used for token counting, utilizing its specific tokenizer to measure token usage accurately.

- **`include_system=True`**  
  When this parameter is set, the system message is preserved during trimming operations.

- **`allow_partial`**  
  Controls whether the last message can be partially truncated to stay within the token limit.  
  In the provided example, this is set to `false`, which removes the entire message if it would exceed the limit, rather than trimming part of it.

- **`start_on="human"`**  
  Ensures that a model’s response (`AIMessage`) is not retained without its corresponding user input (`HumanMessage`).  
  If removal is necessary, both the user message and the AI response are removed together to maintain context.


### Filtering Messages
As the list of chat history messages increases, you may start working with a broader range of message types, subchains, and models.  
To simplify the process of selecting specific messages, LangChain provides the `filter_messages` helper function, which allows filtering messages based on type, ID, or name.

Here’s an example of how to filter only human messages:


In [2]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    filter_messages,
)

messages = [
    SystemMessage("you are a good assistant", id="1"),
    HumanMessage("example input", id="2", name="example_user"),
    AIMessage("example output", id="3", name="example_assistant"),
    HumanMessage("real input", id="4", name="bob"),
    AIMessage("real output", id="5", name="alice"),
]

filter_messages(messages=messages, include_types="human")

[HumanMessage(content='example input', additional_kwargs={}, response_metadata={}, name='example_user', id='2'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='bob', id='4')]

Let's try another example where the filter is used to exclude users and IDs, and include message types:

In [3]:
filter_messages(messages=messages, exclude_names=["example_user", "example_assistant"])

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}, id='1'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='bob', id='4'),
 AIMessage(content='real output', additional_kwargs={}, response_metadata={}, name='alice', id='5')]

In [4]:
filter_messages(
    messages=messages,
    include_types=[HumanMessage, AIMessage],
    exclude_ids = ["3"]
    )

[HumanMessage(content='example input', additional_kwargs={}, response_metadata={}, name='example_user', id='2'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='bob', id='4'),
 AIMessage(content='real output', additional_kwargs={}, response_metadata={}, name='alice', id='5')]

The **filter_messages** function can be used imperatively or declaratively, making it easy to use in cojunction with other component of a chain.

In [5]:
model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21",model="gpt-4o")

filter_ = filter_messages(exclude_names=["example_user", "example_assistant"])

chain = filter_ | model

### Merging Consecutive Messages
Some models, such as Anthropic's chat models, do not support consecutive messages of the same type as input.  
To address this, LangChain provides the `merge_message_runs` utility, which helps by merging consecutive messages of the same type into a single message.


In [6]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    merge_message_runs,
)

messages = [
    SystemMessage("you're a good assistant."),
    SystemMessage("you always respond with a joke."),
    HumanMessage(
        [{"type": "text", "text": "i wonder why it's called langchain"}]
    ),
    HumanMessage("and who is harrison chasing anyway"),
    AIMessage(
        '''Well, I guess they thought "WordRope" and "SentenceString" just 
        didn\'t have the same ring to it!'''
    ),
    AIMessage("""Why, he's probably chasing after the last cup of coffee in the 
        office!"""),
]

merge_message_runs(messages=messages)

[SystemMessage(content="you're a good assistant.\nyou always respond with a joke.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content=[{'type': 'text', 'text': "i wonder why it's called langchain"}, 'and who is harrison chasing anyway'], additional_kwargs={}, response_metadata={}),
 AIMessage(content='Well, I guess they thought "WordRope" and "SentenceString" just \n        didn\'t have the same ring to it!\nWhy, he\'s probably chasing after the last cup of coffee in the \n        office!', additional_kwargs={}, response_metadata={})]

This utility can also be used imperatively or declaratively, making it easy to compose with other components in a chain:

In [7]:
model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21",model="gpt-4o")
merger = merge_message_runs()
chain = merger | model

Let's go back to the [main file](../README.md/#Cognitive-Architectures-with-LangGraph).